In [1]:
import numpy as np
import pandas as pd
import csv, os, re, time, logging, cv2, torch
from pathlib import Path
from importlib.metadata import version
from facenet_pytorch import InceptionResnetV1, MTCNN
from PIL import Image
from moviepy.editor import VideoFileClip
import pytz
import datetime
from win32com.propsys import propsys, pscon
import subprocess
import json
from datetime import datetime
import glob

In [2]:
PYCHARM_DEBUG=True

In [3]:
def print_table(table):
    for row in table:
        print(row)

def create_II_table(tab1, tab2, tab3):
    result_tab = []

    for row1, row2, row3 in zip(tab1, tab2, tab3):
        sum_rows = [int(a) + int(b) + int(c) for a, b, c in zip(row1[1:], row2[1:], row3[1:])]

        total_sum = sum(sum_rows)
        if total_sum > 0:
            # Calculate the percentage distribution
            percentages = [round((x / 3) * 100, 2) for x in sum_rows]
        else:
            percentages = [0] * len(sum_rows)

        result_tab.append([row1[0]] + percentages)

    return result_tab

def create_I_table(tab1, tab2, tab3):
    result_tab = []
    
    emotions = ['Unknown', 'Happy', 'Sad', 'Scared', 'Disgusted', 'Surprised', 'Angry']
    
    for row1, row2, row3 in zip(tab1, tab2, tab3):
        sum_rows = [int(a) + int(b) + int(c) for a, b, c in zip(row1[1:], row2[1:], row3[1:])]
        
        max_sum = max(sum_rows[1:], default=0)
        index = sum_rows.index(max_sum) if max_sum > 1 else -1  # Indeks od 1, jeśli suma jest większa niż 0, w przeciwnym razie -1
        
        if 0 <= index < len(emotions):
            result_tab.append([row1[0], emotions[index]])
        else:
            result_tab.append([row1[0], 'None'])
    return result_tab

# Makes table equal
def fill_table(table, csvreader):
    seconds = 0
    stop = 0
    for row in csvreader:
        row[0] = int(row[0].split('.')[0])
        if seconds <= row[0] and stop == 0:  #fill table with missing seconds
            for i in range(0, row[0]):
                table.append([i, '0', '0', '0', '0', '0', '0', '0'])
                seconds += 1
            stop = 1
            table.append(row)
        else:
            table.append(row)


# Create percentages for every second
def csv_files_reader(base_path):
    index = ['I', 'II', 'III']
    tab1 = []
    tab2 = []
    tab3 = []

    if os.path.exists(base_path):

        for i in index:
            file = os.path.join(base_path, i)
            for file_name in os.listdir(file):
                file_path = os.path.join(file, file_name)

                with open(file_path, 'r') as file:
                    csvreader = csv.reader(file)
                    header = next(csvreader)

                    if i == 'I':
                        fill_table(tab1, csvreader)
                    elif i == 'II':
                        fill_table(tab2, csvreader)
                    elif i == 'III':
                        fill_table(tab3, csvreader)
    else:
        print(f"File path {base_path} doesn't exist.")

    large_table = max([tab1, tab2, tab3], key=len)

    for tab in [tab1, tab2, tab3]:
        if len(tab) < len(large_table):
            diff = len(large_table) - len(tab)

            if len(tab) == 0:
                sec = 0
                for sec in range(len(large_table)):
                    tab.append([sec, '0', '0', '0', '0', '0', '0', '0'])
            else:
                sec = tab[-1][0]  #latest second in table
                for sec in range(tab[-1][0] + 1, len(large_table)):
                    tab.append([sec, '0', '0', '0', '0', '0', '0', '0'])

    return create_I_table(tab1, tab2, tab3), create_II_table(tab1, tab2, tab3)


def get_boris_vectors(path):
    table_I, table_II = csv_files_reader(path)
    df1 = pd.DataFrame(table_I)
    df2 = pd.DataFrame(table_II)
    print(df1)
    print(df2)
    return df1, df2

In [4]:
def get_video_metadata(file_path):
    try:
        # Get the file status
        file_stat = os.stat(file_path)

        # Get the last modified time
        last_modified_time_str = time.ctime(file_stat.st_mtime)
        last_modified_timestamp = int(file_stat.st_mtime)

        # Get the duration of the video
        with VideoFileClip(file_path) as video:
            duration = video.duration

        metadata = {
            'last_modified_time': last_modified_time_str,
            'last_modified_timestamp': last_modified_timestamp,
            'duration': int(duration),  # Duration in seconds
            'initial_timestamp' : last_modified_timestamp - int(duration)
        }
        return metadata

    except FileNotFoundError:
        return f"The file {file_path} does not exist."
    except Exception as e:
        return f"An error occurred: {str(e)}"


In [5]:
def gut_timestamp(video_path):
    return get_video_metadata(video_path)['initial_timestamp']

In [6]:
def ituyu_timestamp(video_path):
    try:
        properties = propsys.SHGetPropertyStoreFromParsingName(video_path)
        timestamp = int(properties.GetValue(pscon.PKEY_Media_DateEncoded).GetValue().timestamp())
        return timestamp
    
    except Exception as e:
        print(f"Error retrieving video metadata: {e}")
        return None

In [7]:
def maap_timestamp(video_path):
    try:
        # Use ffprobe to extract video metadata
        result = subprocess.run(
            ['ffprobe.exe', '-v', 'quiet', '-print_format', 'json', '-show_entries', 'format_tags=creation_time', video_path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True
        )
        metadata = json.loads(result.stdout)
        
        # Extract the creation time if available
        creation_time = metadata.get('format', {}).get('tags', {}).get('creation_time', None)
        
        if creation_time:
            creation_datetime = datetime.fromisoformat(creation_time.replace('Z', '+00:00'))
            unix_timestamp = int(creation_datetime.timestamp())
            return unix_timestamp
        else:
            return None
    
    except Exception as e:
        print(f"Error retrieving video metadata: {e}")
        return None

In [8]:
def get_video_recording_timestamp(video_path, research_center):
    timestamp = 0
    if research_center == 'GUT':
        timestamp = gut_timestamp(video_path)
    elif research_center == 'ITU-YU':
        timestamp = ituyu_timestamp(video_path)
    elif research_center == 'MAAP':
        timestamp = maap_timestamp(video_path)
    return timestamp

In [9]:
# Get the moment of the start of the vector and its frequency
def get_unix_and_hz(file_path):
    with open(file_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        unix = int(float(next(reader)[0]))  # Convert the first cell to integer
        hz = int(float(next(reader)[0]))  # Convert the second cell to integer
    return unix, hz

In [10]:
# Trim the vector to be the multiple of its frequency
def trim_vector(vector, rate):
    length = len(vector)
    if length % rate != 0:
        # Calculate how many elements need to be removed
        excess_elements = length % rate
        # Trim the vector
        vector = vector[:-excess_elements]
    return vector

In [11]:
# Change the frequency by averaging the values
def mean_of_values(vector, rate):
    # Ensure the length of the vector is a multiple of n
    if len(vector) % rate != 0:
        raise ValueError("Length of the vector must be a multiple of frequency")

    # Reshape the vector into a 2D array where each row is a group of n elements
    reshaped_vector = np.reshape(vector, (-1, rate))

    # Calculate the mean along the rows
    mean_values = np.mean(reshaped_vector, axis=1)

    return mean_values

In [12]:
def extract_frame_embeddings(path):
    # Step 1: Initialize FaceNet model and MTCNN detector
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    mtcnn = MTCNN(keep_all=True, device=device)  # MTCNN for face detection
    facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)  # Pre-trained FaceNet model
    
    # Step 2: Load video file and get frame rate
    cap = cv2.VideoCapture(path)

    if not cap.isOpened():
        print("Error: Could not open video.")
    else:
        print("Video opened successfully!")

    # Calculate frames per second (fps) and total frames
    frame_interval = cap.get(cv2.CAP_PROP_FPS)  # Frames per second of the video

    # Step 3: Process video at 1 second intervals
    frame_count = 0
    frame_embeddings = []
    num_of_features = 512

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Check if the frame is at the 1-second interval
        if frame_count % frame_interval < 1:
            # Convert frame to RGB (OpenCV uses BGR by default)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            try:
                # Step 4: Detect faces in the frame
                boxes, _ = mtcnn.detect(frame_pil)
                if boxes is not None and len(boxes) > 0:
                    # Step 5: Find the smallest box based on area
                    areas = [(box[2] - box[0]) * (box[3] - box[1]) for box in boxes]
                    smallest_box = boxes[np.argmin(areas)]
    
                    x1, y1, x2, y2 = [int(coord) for coord in smallest_box]
                    cropped_face = frame_rgb[y1:y2, x1:x2]
                    cropped_face_pil = Image.fromarray(cropped_face)
    
                    # Step 6: Crop the face from the frame
                    faces = mtcnn(cropped_face_pil)
                    if faces is not None and len(faces) > 0:
                        faces = faces.to(device)
                        embeddings = facenet(faces)  # Generate embeddings
                        frame_embeddings.append(embeddings.detach().cpu().numpy())
                        #print("Successful.")
                    else:
                        # Append zeros if no valid face tensor detected
                        frame_embeddings.append(np.zeros(num_of_features, dtype=np.float32))
                        #print("Failed after cropping.")
                else:
                    frame_embeddings.append(np.zeros(num_of_features, dtype=np.float32))
                    #print("No faces detected")
                    
            except Exception as e:
                # Handle any exception raised by MTCNN and append zeros
                #print(f"Exception during face detection: {e}")
                frame_embeddings.append(np.zeros(num_of_features, dtype=np.float32))

        frame_count += 1

    # Step 7: Release video capture
    cap.release()
    data = np.vstack(frame_embeddings)
    return pd.DataFrame(data)

In [13]:
def longer_video(dir_path):
    mp4_file_path = glob.glob(os.path.join(dir_path, "*.mp4"))[0]
    mts_file_path = glob.glob(os.path.join(dir_path, "*.MTS"))[0]
    # Get the duration of the video
    with VideoFileClip(mp4_file_path) as video:
        mp4_duration = video.duration
        
    with VideoFileClip(mts_file_path) as video:
        mts_duration = video.duration
    
    if mp4_duration > mts_duration:
        return re.compile(r'^Untitled \d+\.mp4$')
    else:
        return re.compile(r'^\d{5}\.MTS$')

In [17]:
def slice_vectors(video_path, biosignal_path, input_storage_path, boris1, boris2, boris1_storage_path, boris2_storage_path, research_center):
    # Get the path for EDA, TEMP and HR
    EDA_path = os.path.join(biosignal_path, 'EDA.csv')
    TEMP_path = os.path.join(biosignal_path, 'TEMP.csv')
    HR_path = os.path.join(biosignal_path, 'HR.csv')
    
    # Get only the data
    EDA = pd.read_csv(EDA_path, skiprows = 2, header = None)
    TEMP = pd.read_csv(TEMP_path, skiprows = 2, header = None)
    HR = pd.read_csv(HR_path, skiprows = 2, header = None)
    frame_embeddings = extract_frame_embeddings(video_path)
    
    # Get the starting time and frequency
    unix_EDA, hz_EDA = get_unix_and_hz(EDA_path)
    unix_TEMP, hz_TEMP = get_unix_and_hz(TEMP_path)
    unix_HR, hz_HR = get_unix_and_hz(HR_path)
    unix_video = get_video_recording_timestamp(video_path, research_center)

    # Unificate the frequencies  
    EDA_mean = mean_of_values(trim_vector(EDA, hz_EDA), hz_EDA)
    TEMP_mean = mean_of_values(trim_vector(TEMP, hz_TEMP), hz_TEMP)
    HR_mean = mean_of_values(trim_vector(HR, hz_HR), hz_HR)

    # Get the lengths
    length_EDA = len(EDA_mean)
    length_TEMP = len(TEMP_mean)
    length_HR = len(HR_mean)
    length_video = get_video_metadata(video_path)['duration']
    
    # Get the vectors of starts and ends for biosignals and video
    starts = [unix_HR, unix_TEMP, unix_EDA, unix_video]
    ends = [unix_HR + length_HR, unix_TEMP + length_TEMP, unix_EDA + length_EDA, unix_video + length_video]

    # Get the latest start of any vector
    last_start = max(starts)
    
    # Get the earliest end of any vector
    first_end = min(ends)

    # Get matching indexes for start and end for every vector
    EDA_first_index = last_start - unix_EDA
    EDA_last_index = first_end - unix_EDA
    TEMP_first_index = last_start - unix_TEMP
    TEMP_last_index = first_end - unix_TEMP
    HR_first_index = last_start - unix_HR
    HR_last_index = first_end - unix_HR
    video_first_index = last_start - unix_video
    video_last_index = first_end - unix_video

    print('EDA: ', EDA_last_index - EDA_first_index)
    print('TEMP: ', TEMP_last_index - TEMP_first_index)
    print('HR: ', HR_last_index - HR_first_index)
    print('video: ', video_last_index - video_first_index)
    
    # Slice the biosignals based on the index
    sliced_EDA = EDA[EDA_first_index:EDA_last_index]
    sliced_TEMP = TEMP[TEMP_first_index:TEMP_last_index]
    sliced_HR = HR[HR_first_index:HR_last_index]
    sliced_boris_1 = boris1[video_first_index:video_last_index].copy() # to surpass the warning of working on a view and not a copy
    sliced_boris_2 = boris2[video_first_index:video_last_index].copy() # to surpass the warning of working on a view and not a copy
    sliced_frame_embeddings = frame_embeddings[video_first_index:video_last_index]
    
    # Create one df with all biosignals
    input = pd.concat([sliced_EDA.reset_index(drop=True), sliced_TEMP.reset_index(drop=True), sliced_HR.reset_index(drop=True), sliced_frame_embeddings.reset_index()], axis=1)
    
    # Rename column names
    new_columns = ['EDA', 'TEMP', 'HR'] + input.columns[3:].tolist()
    input.columns = new_columns
    input = input.drop(input.columns[3], axis=1)
    # Drop index and contempt from BORIS
    sliced_boris_1.drop([sliced_boris_1.columns[0]], axis=1, inplace=True)
    sliced_boris_2.drop([sliced_boris_2.columns[0], sliced_boris_2.columns[1]], axis=1, inplace=True)
    sliced_boris_2.columns = ['Happy', 'Sad', 'Scared', 'Disgusted', 'Surprised', 'Angry']
    # Add the biosignals to file
    input.to_csv(input_storage_path, index=False, sep=',')
    sliced_boris_1.to_csv(boris1_storage_path, index=False, sep=',')
    sliced_boris_2.to_csv(boris2_storage_path, index=False, sep=',')

In [19]:
def traverse():
    root_dir = '//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE'
    target_root_dir = '//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis'
    boris_ext = 'Analysis/BORIS/'
    type = ['Camera', 'Wristband']
    research_centers = ['GUT', 'ITU-YU', 'MAAP']
    s_values = ["S01", "S02", "S03", "S04", "S05", "S06", "S07", "S08", "S09", "S10", "S11" ]
    c_values = ["C01", "C02", "C03", "C04", "C05", "C06", "C07", "C08", "C09", "C10", "C11", "C12", "C13" ]
    for research_center in research_centers:
        for session in s_values:
            # Create the session path for biosignal and camera + BORIS
            camera_path = Path(root_dir).joinpath(type[0], research_center, session)
            signals_path = Path(root_dir).joinpath(type[1], research_center, session)
            if camera_path.is_dir() and signals_path.is_dir():
                # Create the meeting path for biosignal and camera + BORIS
                for camera in c_values:
                    exact_camera_path = Path(camera_path).joinpath(camera)
                    exact_signals_path = Path(signals_path).joinpath(camera)
                    if exact_camera_path.is_dir() and exact_signals_path.is_dir():
                        boris_path = Path(exact_camera_path).joinpath(boris_ext)
                        if boris_path.is_dir():
                            # Quick fix of two MAAP sessions being divided TODO: Fix this
                            if not(research_center == 'MAAP' and ((session == 'S01' and camera == 'C05') or (session == 'S03' and camera == 'C05'))):
                                mp4_pattern = []
                                if research_center == 'GUT':
                                    video = longer_video(exact_camera_path)
                                    mp4_pattern.append(video)
                                elif research_center == 'ITU-YU':
                                    mp4_pattern.append(re.compile(r'^ITU-C\d{2}-S\d{2}-\d{8}-Camera\.mp4$'))
                                elif research_center == 'MAAP':
                                    mp4_pattern.append(re.compile(r'^MAAP-C\d{2}-S\d{2}-\d{8}-Camera\.AVI$'))
                                    mp4_pattern.append(re.compile(r'^MAAP-C\d{2}-S\d{2}-\d{8}-Camera\.MOV$'))
                                for files_in_camera_dir in os.listdir(exact_camera_path):
                                    if any(pattern.match(files_in_camera_dir) for pattern in mp4_pattern):
                                        video_path = os.path.join(exact_camera_path, files_in_camera_dir)
                                # TODO: TRY EXCEPT FOR not matching pattern, skip iteration
                                filename = research_center + '_' + session + '_' + camera
                                boris1_filename = filename + '_BORIS_method_I.csv'
                                boris2_filename = filename + '_BORIS_method_II.csv'
                                boris1_target_dir = Path(target_root_dir).joinpath(research_center, boris1_filename)
                                boris2_target_dir = Path(target_root_dir).joinpath(research_center, boris2_filename)
                                input_filename = filename + '_input.csv'
                                input_target_dir = Path(target_root_dir).joinpath(research_center, input_filename)
                                # Create combined BORIS vector
                                boris1, boris2 = get_boris_vectors(boris_path)
                                # Create one vector of biosignals, sliced BORIS and sliced video
                                slice_vectors(video_path, exact_signals_path, input_target_dir, boris1, boris2, boris1_target_dir, boris2_target_dir, research_center)
traverse()

       0     1
0      0  None
1      1  None
2      2  None
3      3  None
4      4  None
..   ...   ...
605  605  None
606  606  None
607  607  None
608  608  None
609  609  None

[610 rows x 2 columns]
       0    1    2    3    4    5    6    7
0      0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1      1  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2      2  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3      3  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4      4  0.0  0.0  0.0  0.0  0.0  0.0  0.0
..   ...  ...  ...  ...  ...  ...  ...  ...
605  605  0.0  0.0  0.0  0.0  0.0  0.0  0.0
606  606  0.0  0.0  0.0  0.0  0.0  0.0  0.0
607  607  0.0  0.0  0.0  0.0  0.0  0.0  0.0
608  608  0.0  0.0  0.0  0.0  0.0  0.0  0.0
609  609  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[610 rows x 8 columns]
EDA:  546
TEMP:  546
HR:  546
video:  546
       0     1
0      0  None
1      1  None
2      2  None
3      3  None
4      4  None
..   ...   ...
572  572  None
573  573  None
574  574  None
575  575  None
576  576  None

[577 rows x 2 columns